In [ ]:
! pip install langchain==0.1.14 openai==1.14.3 langchain-openai==0.1.1
! pip install pypdf
! pip install langchain-chroma

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import time

class RAGBookClubGPT:
    def __init__(self, pdf_path, api_key):
        """
        RAG 기반 독서 모임 AI 초기화
        pdf_path: 독서 모임 관련 PDF 문서 경로
        api_key: OpenAI API 키
        """
        # 데이터 로드
        loader = PyPDFLoader(pdf_path)
        self.docs = loader.load()

        # 벡터 임베딩 생성 및 데이터베이스 구축
        embeddings_model = OpenAIEmbeddings(api_key=api_key)
        self.db = Chroma.from_documents(self.docs, embeddings_model)

        # 데이터 검색기 및 QA 체인 초기화
        self.retriever = self.db.as_retriever()
        self.llm = ChatOpenAI(model_name="gpt-4", api_key=api_key)
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.retriever
        )

    def generate_topics(self, user_responses):
        """
        RAG 기반으로 발제문 생성
        user_responses: 참가자 감상문
        """
        system_message = "Give me three topics in Korean that I can freely discuss about this book. They can't be political or hateful. They can't have set answers."
        user_message = "\n".join(user_responses)

        # 질의에 대해 RAG 기반 문맥 검색 및 발제문 생성
        context_docs = self.retriever.get_relevant_documents("발제문 생성에 필요한 주요 내용")
        context_text = "\n".join(doc.page_content for doc in context_docs)

        prompt = f"{system_message}\n\nPDF 문서 내용:\n{context_text}\n\n참가자 감상문:\n{user_message}"
        response = self.llm.predict(prompt)

        topics = response.split("\n")  # 발제문 나누기
        print("선정된 발제문:")
        for topic in topics:
            print(f"- {topic}")
        return topics

    def collect_responses(self, topic):
        """
        주어진 주제에 대해 참가자들의 의견을 수집하고, 욕설 및 비방 내용을 필터링하는 메서드
        """
        print(f'\n주제: {topic}')  # 현재 토론 주제를 출력
        print('이 주제에 대해 의견을 작성해 주세요.')  # 사용자에게 의견 작성 요청

        # 실제 사용자 입력을 받는 부분
        responses = []
        while True:
            user_input = input("사용자 입력 (형식: 사용자명: 의견, 종료 시 '종료' 입력): ").strip()
            if user_input.lower() == "종료":
                break

            # 사용자명과 의견을 분리
            try:
                user, response = user_input.split(":", 1)
                responses.append(f"{user.strip()}: {response.strip()}")
            except ValueError:
                print("잘못된 입력 형식입니다. '사용자명: 의견' 형식으로 입력해주세요.")

        # 필터링된 응답 처리 (욕설 및 비방 내용 제거)
        filtered_responses = self.filter_responses(responses)

        # 필터링된 의견 출력
        print('필터링된 의견:')
        for response in filtered_responses:
            print(response)

        return filtered_responses  # 필터링된 응답 반환

    def filter_responses(self, responses):
        """
        참가자들의 응답에서 욕설이나 비방이 포함된 내용을 필터링하는 메서드.
        """
        bad_words = ['욕설', '비방']  # 필터링할 단어 목록
        filtered = []  # 필터링된 응답을 저장할 리스트

        for response in responses:
            # 응답에 필터링할 단어가 포함되지 않은 경우만 리스트에 추가
            if not any(bad_word in response for bad_word in bad_words):
                filtered.append(response)

        return filtered  # 필터링된 응답 리스트 반환

    def free_discussion(self):
        """
        자유 토론 시간을 제공하는 메서드. 실제로는 10분 동안 대기하지만, 테스트를 위해 짧게 설정됨.
        """
        print('이제 자유 토의 시간입니다. 10분 동안 의견을 나누세요.')

        # 실제로는 time.sleep(600)으로 10분 대기해야 하지만, 테스트를 위해 짧게 설정함.
        time.sleep(1)  # 테스트용 대기 시간

        print('자유 토의 시간 종료.')  # 자유 토론 종료 알림

    def summarize_meeting(self, topics, all_responses):
        """
        RAG 기반으로 모임 내용을 요약하는 메서드
        topics: 논의된 주제들
        all_responses: 참가자들의 모든 응답
        """
        print('\n모임 내용을 요약합니다...')

        # GPT-4o에게 전달할 요약 요청 프롬프트 생성 (참가자들의 논의 내용 포함)
        summary_prompt = "다음은 독서 모임에서 논의된 내용입니다. 이 내용을 바탕으로 간단한 요약을 작성해줘:\n"

        for i, topic in enumerate(topics):
            summary_prompt += f"주제 {i + 1}: {topic}\n"
            summary_prompt += "응답:\n" + "\n".join(all_responses[i]) + "\n"

        # GPT-4o API 호출하여 요약 생성 (RAG 기반)
        response = self.llm.predict(summary_prompt)

        summary = response.strip()
        print("모임 요약:")  # 요약 출력
        print(summary)


# PDF 경로 및 OpenAI API 키
pdf_path = "/content/현진건-운수좋은날+B3356-개벽.pdf"
api_key = "##API key##"

# RAGBookClubGPT 인스턴스 생성
book_club_ai = RAGBookClubGPT(pdf_path, api_key)

# 참가자 감상문
user_responses = [
    "현진건의 운수 좋은 날은 일제강점기 하층민의 비극적 삶을 그린 작품으로, 김첨지라는 인력거꾼의 하루를 통해 아이러니와 비극을 극대화한다. 김첨지는 아픈 아내의 만류에도 불구하고 일을 나가고, 그날따라 유난히 운이 좋아 많은 돈을 번다. 하지만 집에 돌아와서는 이미 싸늘한 시신이 된 아내를 발견하게 된다. 작품 제목에서부터 드러나는 반어적 표현은 독자에게 깊은 여운을 남긴다. 설렁탕을 사 왔지만 아내가 먹지 못하는 상황은 당시 하층민의 절망적인 현실을 상징적으로 보여준다. 이 작품은 단순한 개인의 비극을 넘어, 일제강점기라는 시대적 배경 속에서 민중이 겪어야 했던 고통과 무력함을 잘 드러낸다.",
    "운수 좋은 날은 겉으로는 운이 좋은 하루였지만, 결국에는 비극으로 끝나는 김첨지의 이야기를 통해 삶의 아이러니를 잘 보여준다. 김첨지는 그날따라 많은 돈을 벌어 기뻐하지만, 집에 돌아와 아내의 죽음을 맞닥뜨린다. 이 작품은 당시 사회적 현실과 개인의 비극이 어떻게 얽혀 있는지를 심도 있게 탐구한다. 특히, 김첨지가 아내를 위해 사 온 설렁탕이 결국 아무 의미 없게 되는 장면은 독자의 마음을 더욱 아프게 한다. 이러한 아이러니는 작품의 비극성을 한층 더 강화하며, 독자로 하여금 당시 시대 상황에 대한 깊은 성찰을 유도한다",
    "현진건의 운수 좋은 날은 일제강점기 하층민의 비극적 삶을 그린 작품으로, 김첨지라는 인력거꾼의 하루를 통해 아이러니와 비극을 극대화한다. 김첨지는 아픈 아내의 만류에도 불구하고 일을 나가고, 그날따라 유난히 운이 좋아 많은 돈을 번다. 하지만 집에 돌아와서는 이미 싸늘한 시신이 된 아내를 발견하게 된다. 작품 제목에서부터 드러나는 반어적 표현은 독자에게 깊은 여운을 남긴다. 설렁탕을 사 왔지만 아내가 먹지 못하는 상황은 당시 하층민의 절망적인 현실을 상징적으로 보여준다. 이 작품은 단순한 개인의 비극을 넘어, 일제강점기라는 시대적 배경 속에서 민중이 겪어야 했던 고통과 무력함을 잘 드러낸다."
]

# 발제문 생성 및 참가자 의견 수집
topics = book_club_ai.generate_topics(user_responses)

# 각 주제별로 참가자 의견 수집
all_responses = []
for topic in topics:
    responses = book_club_ai.collect_responses(topic)
    all_responses.append(responses)

# 자유 토론
book_club_ai.free_discussion()

# 모임 요약
book_club_ai.summarize_meeting(topics, all_responses)


선정된 발제문:
- 1. 김첨지의 삶을 통해 보는 일제강점기 하층민의 삶과 고통
- 2. '운수 좋은 날'에서 나타나는 아이러니와 비극성에 대한 이해
- 3. 운수 좋은 날에서 사용된 상징과 비유, 그리고 그것이 주는 메시지와 의미에 대해

주제: 1. 김첨지의 삶을 통해 보는 일제강점기 하층민의 삶과 고통
이 주제에 대해 의견을 작성해 주세요.
사용자 입력 (형식: 사용자명: 의견, 종료 시 '종료' 입력): 사용자1: 김첨지는 인력거꾼으로서 하루 벌어 하루 먹고 사는 삶을 살아갑니다. 그의 경제적 상황은 매우 불안정하며, 열흘 동안 돈을 벌지 못하는 상황도 흔합니다. 이러한 경제적 궁핍은 그가 아내의 병세에도 불구하고 일을 나갈 수밖에 없게 만듭니다16. 이는 당시 하층민들이 겪었던 경제적 불안정성을 상징적으로 보여줍니다
사용자 입력 (형식: 사용자명: 의견, 종료 시 '종료' 입력): 사용자2: 김첨지의 아내는 병들어 있지만, 가난 때문에 제대로 된 치료를 받지 못하고 결국 죽음에 이릅니다. 이는 가난이 질병을 낳고, 질병이 다시 가난을 심화시키는 악순환을 나타냅니다3. 이러한 상황은 당시 하층민들이 직면한 현실적인 문제였으며, 사회적 안전망의 부재를 여실히 드러냅니다.
사용자 입력 (형식: 사용자명: 의견, 종료 시 '종료' 입력): 사용자3: 김첨지와 그의 가족은 사회로부터 철저히 소외된 존재로 그려집니다. 그들의 고통과 비극은 개인적인 문제로 치부되며, 사회적 지원이나 관심은 전혀 없습니다5. 이는 일제강점기 하층민들이 겪었던 사회적 소외와 무관심을 반영합니다.
사용자 입력 (형식: 사용자명: 의견, 종료 시 '종료' 입력): 사용자4: 작품에서 김첨지는 운이 좋은 날이라며 돈을 많이 벌지만, 집에 돌아와서는 아내의 죽음을 맞이하게 됩니다. 이러한 아이러니는 당시 하층민들의 삶이 얼마나 비극적이고 예측 불가능했는지를 보여줍니다24. 이는 개인의 노력만으로는 극복할 수 없는 구조적 문제를 암시하며, 독자에게 깊은 여운을 남깁니다.
사용자 입력 (형식: